# 多线程

## 多线程的含义

首先需要了解什么是进程

进程：一个可以独立运行的程序单元。比如打开一个浏览器，就是打开了一个浏览器进程；但是一个进程同时可以处理很多事情，比如一个浏览器中可以打开多个页面，多个页面可以同时运行，互不干扰。

为什么可以同时运行多个任务呢？这就引出了线程的概念了，其实这一个个任务，就对应着一个个线程的执行。

而进程就是线程的集合，进程就是由多个线程构成的，线程是操作系统进行运算测度的最小单位，是进程中的一个最小运行单元。

多线程就是一个进程中同时执行多个线程，前面浏览器的情景就是典型的多线程执行。

## 并发和并行

说到多线程和多进程，还需要讲解一下并发和并行。

### 并发(concurrency)

同一时刻只能有一条指令执行，但是多个线程的对应指令被快速轮换地执行。比如一个处理器在线程A,B之间快速切换执行，但是执行和切换速度之快，人感知不到多个线程的轮换执行。所以**宏观上看起来多个线程在同时运行，但是微观上是处理器在不同的线程之间切换和执行**

### 并行(parallel)

同一时刻，有多条指令在多个处理器上同时执行，并行必须依赖多个处理器。如果我们的计算机处理器只有一个核，那它只能通过并发的方式运行这些线程。如果有多个核，当一个核在执行一个线程时，另一个核可以执行另一个线程，这样两个线程就实现了并行运行。

## 多线程适用场景

在一个程序执行过程中，有些操作时比较耗时和需要等待的（例如等待数据库的查询结果），这时如果使用单线程，处理器必须等到这些操作完成后才能继续向下执行其他操作。如果使用多线程，就可以在等待某个线程时，去执行其他线程，从而整体上提高执行效率。

网络爬虫也是一个经典的需要多线程的例子：爬虫在向服务器发送请求后，需要等待服务器的响应，这种IO密集型任务可以启用多线程，这样可以在等待某个线程时去处理其他任务。从而大大提高整体的爬取效率。

还有一种任务叫做计算密集型，也就是CPU密集型任务，任务的运行一直需要处理器的参与，那么使用多线程反而会增加切换线程的时间，整体效率反而变低。

## Python实现多线程

Python实现多线程的模块叫做threading。

### thread直接创建多线程

In [9]:
import threading
import time 

def target(second):
    # 线程的名字就是通过threading.current_thread().name获得，主线程的名字为MainThreat，子线程的名字为Thread-num
    print(f'threat {threading.current_thread().name} is running')
    print(f'threat {threading.current_thread().name} sleep {second}s')
    time.sleep(second)
    print(f'threat {threading.current_thread().name} is ended')

print(f'Threading {threading.current_thread().name} is running')
for i in [1,5]:
    # 使用threading.Tread()类新建了两个线程，target参数就是刚才定义的target函数，args通过列表的形式传递
    thread = threading.Thread(target=target,args=[i])
    # 新建线程声明之后，使用start()方法即可开始线程的运行
    thread.start()
print(f'Threading {threading.current_thread().name} is ended')

Threading MainThread is running
threat Thread-18 is running
threat Thread-18 sleep 1s
threat Thread-19 is running
threat Thread-19 sleep 5sThreading MainThread is ended

threat Thread-18 is ended
threat Thread-19 is ended


Threading MainThread is running

threat Thread-1 is running

threat Thread-1 sleep 1s

threat Thread-2 is running

threat Thread-2 sleep 5s

Threading MainThread is ended

threat Thread-1 is ended

threat Thread-2 is ended

上面是代码运行应该得到的输出（在jupyter看不到threat1、2的end？）。可以看出，主线程首先结束，接着Threat-1和Threat-2才结束，并且分别间隔了1s和4s。说明主线程并没有等待子线程运行完毕才结束，而是直接结束。

如果我们想主线程等待子线程运行完毕才推出，可以让每个子线程对线都调用join方法，实现如下：

In [10]:
threads = []
for i in [1,5]:
    thread = threading.Thread(target=target,args=[i])
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()

threat Thread-20 is running
threat Thread-20 sleep 1s
threat Thread-21 is running
threat Thread-21 sleep 5s
threat Thread-20 is ended
threat Thread-21 is ended


## 继承Thread类创建子线程

通过继承Thread类的方式创建一个线程，该线程需要执行的方法写在类的run方法里面即可。上面的例子可等价写成：



In [11]:
import threading
import time 

class MyThread(threading.Thread):
    def __init__(self,second):
        threading.Thread.__init__(self)
        self.second = second
    
    def run(self):
        print(f'Threading {threading.current_thread().name} is running')
        print(f'Threading {threading.current_thread().name} sleep {self.second}s')
        time.sleep(self.second)
        print(f'Threading {threading.current_thread().name} is ended')

print(f'Threading {threading.current_thread().name} is running')
threads=[]
for i in [1,5]:
    thread = MyThread(i)
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
print(f'Threading {threading.current_thread().name} is ended')

Threading MainThread is running
Threading Thread-22 is running
Threading Thread-22 sleep 1s
Threading Thread-23 is running
Threading Thread-23 sleep 5s
Threading Thread-22 is ended
Threading Thread-23 is ended
Threading MainThread is ended


## 守护线程

如果一个线程被设置为守护线程，那么意味着这个线程是“不重要”的，如果主线程结束了而该守护线程还没有运行完，那么它将会被强制结束

In [12]:
print(f'Threading {threading.current_thread().name} is running')
t1 = MyThread(1)
t1.start()
t2 = MyThread(5)
# 将threat2设置为守护线程 setDaemon
t2.setDaemon(True)
t2.start()
print(f'Threading {threading.current_thread().name} is ended')

Threading MainThread is running
Threading Thread-24 is running
Threading Thread-24 sleep 1s
Threading Thread-25 is running
Threading Thread-25 sleep 5s
Threading MainThread is ended


在这里我们通过 setDaemon 方法将 t2 设置为了守护线程，这样主线程在运行完毕时，t2 线程会随着线程的结束而结束。

这里并没有调用 join 方法，如果我们让 t1 和 t2 都调用 join 方法，主线程就会仍然等待各个子线程执行完毕再退出，不论其是否是守护线程。

## 互斥锁

在一个进程中，多个线程是共享资源的，比如在一个进程中，有一个全局变量count，现在我们声明多个线程，每个线程运行时都给count加一。


In [17]:
count = 0
class MyThread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)

    def run(self):
        global count
        temp = count + 1
        time.sleep(0.001)
        count = temp 

threads = []
for _ in range(1000):
    thread = MyThread()
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()
print(f'Final count: {count}')

Final count: 20


因为 count 这个值是共享的，每个线程都可以在执行 temp = count 这行代码时拿到当前 count 的值，但是这些线程中的一些线程可能是并发或者并行执行的，这就导致不同的线程拿到的可能是同一个 count 值，最后导致有些线程的 count 的加 1 操作并没有生效，导致最后的结果偏小。

所以，如果多个线程同时对某个数据进行读取或修改，就会出现不可预料的结果。为了避免这种情况，我们需要对多个线程进行同步，要实现同步，我们可以对需要操作的数据进行加锁保护，这里就需要用到 threading.Lock 了。

加锁保护是什么意思呢？就是说，某个线程在对数据进行操作前，需要先加锁，这样其他的线程发现被加锁了之后，就无法继续向下执行，会一直等待锁被释放，只有加锁的线程把锁释放了，其他的线程才能继续加锁并对数据做修改，修改完了再释放锁。这样可以确保同一时间只有一个线程操作数据，多个线程不会再同时读取和修改同一个数据，这样最后的运行结果就是对的了。

In [19]:
count = 0
class MyThread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)

    def run(self):
        global count
        lock.acquire()
        temp = count + 1
        time.sleep(0.001)
        count = temp 
        lock.release()
        
lock = threading.Lock()
threads = []
for _ in range(1000):
    thread = MyThread()
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()
print(f'Final count: {count}')

Final count: 1000


关于 Python 多线程的内容，这里暂且先介绍这些，关于 theading 更多的使用方法，如信号量、队列等，可以参考官方文档：https://docs.python.org/zh-cn/3.7/library/threading.html#module-threading。

## Python 多线程的问题

由于 Python 中 GIL 的限制，导致不论是在单核还是多核条件下，在同一时刻只能运行一个线程，导致 Python 多线程无法发挥多核并行的优势。

GIL 全称为 Global Interpreter Lock，中文翻译为全局解释器锁，其最初设计是出于数据安全而考虑的。

在 Python 多线程下，每个线程的执行方式如下：

- 获取 GIL

- 执行对应线程的代码

- 释放 GIL

可见，某个线程想要执行，必须先拿到 GIL，我们可以把 GIL 看作是通行证，并且在一个 Python 进程中，GIL 只有一个。拿不到通行证的线程，就不允许执行。这样就会导致，即使是多核条件下，一个 Python 进程下的多个线程，同一时刻也只能执行一个线程。

不过对于爬虫这种 IO 密集型任务来说，这个问题影响并不大。而对于计算密集型任务来说，由于 GIL 的存在，多线程总体的运行效率相比可能反而比单线程更低。

# 多进程

Python中的多线程是不能很好地发挥多核优势的，要想发挥多核优势，最好还是使用多进程。

## 多进程的含义

进程(Process)是具有一定独立功能的程序关于某个数据集合上的一次运行活动，是系统进行资源分配和调度的一个独立单位。

多进程就是多个进程同时进行，由于进程是线程的集合，多进程的运行意味着大于等于进程数量的线程在运行。

## python多进程的优势

对于多进程，每个进程都有自己的GIL，所以多进程的运行是不会受到GIL影响的。因此多进程能更好地发挥多核的优势。当然，对于爬虫这种 IO 密集型任务来说，多线程和多进程影响差别并不大。对于计算密集型任务来说，Python 的多进程相比多线程，其多核运行效率会有成倍的提升。

总的来说，Python 的多进程整体来看是比多线程更有优势的。所以，在条件允许的情况下，能用多进程就尽量用多进程

但是，进程之间的数据无法共享，比如多个进程无法共享同一个全局变量。进程之间的数据共享需要有单独的机制来实现。

## 多进程的实现

Python内置库`multiprocessing`提供了一系列组件:Process, Queue, Semaphore, Pipe, Lock, Pool等。

### 直接使用Process类

在这个库中，每个进程都用一个Process类来表示，其API：

`Process([group [, target [, name [, args [, kwargs]]]]])`

- target 表示调用对象，你可以传入方法的名字。

- args 表示被调用对象的位置参数元组，比如 target 是函数 func，他有两个参数 m，n，那么 args 就传入 [m, n] 即可。

- kwargs 表示调用对象的字典。

- name 是别名，相当于给这个进程取一个名字。

- group 分组。

In [3]:
import multiprocessing

def target(index):
    print(f'Process {index}')

if __name__ == '__main__':
    for i in range(5):
        p = multiprocessing.Process(target=target, args=(i,))
        p.start()

In [4]:
# multiprocessing.cpu_count(),multiprocessing.active_children()

import multiprocessing
import time

def process(index):
    time.sleep(index)
    print(f'Process: {index}')

if __name__ == '__main__':
    for i in range(5):
        p = multiprocessing.Process(target=process, args=[i])
        p.start()
    print(f'CPU number: {multiprocessing.cpu_count()}')
    for p in multiprocessing.active_children():
        print(f'Child process name: {p.name} id: {p.pid}')
    print('Process Ended')

CPU number: 8
Child process name: Process-13 id: 23248
Child process name: Process-12 id: 24404
Child process name: Process-11 id: 18784
Child process name: Process-14 id: 13984
Child process name: Process-15 id: 14812
Process Ended


## 继承Process类

进程的基本操作在子类的run方法中实现即可

In [6]:
class MyProcess(multiprocessing.Process):
    def __init__(self,loop):
        multiprocessing.Process.__init__(self)
        self.loop = loop
    
    def run(self):
        for i in range(self.loop):
            print(f'Pid {self.pid}; Loopcount {i}')

if __name__ == "__main__":
    for i in range(2,5):
        p = MyProcess(i)
        p.start()    

通过上面的方式，可以非常方便的实现进程的定义，为了复用方便，可以把一些方法写在进程类里封装好，在使用时直接初始化一个进程类即可。

## 守护进程

如果一个进程被设置为守护进程，当父进程结束后，子进程会自动被终止，我们可以通过daemon属性来设置是否为守护进程。

In [7]:
from multiprocessing import Process
import time

class MyProcess(Process):
    def __init__(self, loop):
        Process.__init__(self)
        self.loop = loop

    def run(self):
        for count in range(self.loop):
            time.sleep(1)
            print(f'Pid: {self.pid} LoopCount: {count}')

if __name__ == '__main__':
    for i in range(2, 5):
        p = MyProcess(i)
        p.daemon = True
        p.start()

print('Main Process ended')

Main Process ended


结果很简单，因为主进程没有做任何事情，直接输出一句话结束，所以在这时也直接终止了子进程的运行。

这样可以有效防止无控制地生成子进程。这样的写法可以让我们在主进程运行结束后无需额外担心子进程是否关闭，避免了独立子进程的运行。

## 进程等待

上面的运行结果不符合预期，我们能不能让所有的子进程都运行完了再结束？只需要加上join方法。

In [8]:
processes = []
for i in range(2,5):
    p = MyProcess(i)
    processes.append(p)
    p.daemon = True
    p.start()

for p in processes:
    p.join()

在调用 start 和 join 方法后，父进程就可以等待所有子进程都执行完毕后，再打印出结束的结果。

### 超时设置

默认情况下，join 是无限期的。也就是说，如果有子进程没有运行完毕，主进程会一直等待。这种情况下，如果子进程出现问题陷入了死循环，主进程也会无限等待下去。怎么解决这个问题呢？可以给 join 方法传递一个超时参数，代表最长等待秒数。如果子进程没有在这个指定秒数之内完成，会被强制返回，主进程不再会等待。也就是说这个参数设置了主进程等待该子进程的最长时间。

例如这里我们传入 1，代表最长等待 1 秒，代码改写如下：

In [ ]:
for p in processes:
    p.join(1)

## 终止进程

还可以通过terminate方法来终止某个子进程，还可以通过is_alive()方法判断进程是否在运行。

In [9]:
import multiprocessing
import time

def process():
    print('Starting')
    time.sleep(5)
    print('Finished')

if __name__ == '__main__':
    p = multiprocessing.Process(target=process)
    print('Before:', p, p.is_alive())

    p.start()
    print('During:', p, p.is_alive())

    p.terminate()
    print('Terminate:', p, p.is_alive())

    p.join()
    print('Joined:', p, p.is_alive())


Before: <Process(Process-28, initial)> False
During: <Process(Process-28, started)> True
Terminate: <Process(Process-28, started)> True
Joined: <Process(Process-28, stopped[SIGTERM])> False


这里有一个值得注意的地方，在调用 terminate 方法之后，我们用 is_alive 方法获取进程的状态发现依然还是运行状态。在调用 join 方法之后，is_alive 方法获取进程的运行状态才变为终止状态。

所以，在调用 terminate 方法之后，记得要调用一下 join 方法，这里调用 join 方法可以为进程提供时间来更新对象状态，用来反映出最终的进程终止效果。

## 进程互斥锁

```python
Pid: 73993 LoopCount: 0
Pid: 73993 LoopCount: 1
Pid: 73994 LoopCount: 0Pid: 73994 LoopCount: 1

Pid: 73994 LoopCount: 2
Pid: 73995 LoopCount: 0
Pid: 73995 LoopCount: 1
Pid: 73995 LoopCount: 2
Pid: 73995 LoopCount: 3
Main Process ended
```

有的输出没有换行，这是因为多个进程并行执行导致的，两个进程同时输出，导致最终输出没有换行。

如何避免呢？

多个进程运行期间的任意时间，只能有一个进程输出，其他进程等待~~这其实就是进程互斥，避免了多个进程同时抢占临界区（输出）资源。可以通过multiprocessing中的Lock来实现。在一个进程输出时，加锁，其他进程等待。



In [10]:
from multiprocessing import Process, Lock
import time

class MyProcess(Process):
    def __init__(self, loop, lock):
        Process.__init__(self)
        self.loop = loop
        self.lock = lock

    def run(self):
        for count in range(self.loop):
            time.sleep(0.1)
            self.lock.acquire()
            print(f'Pid: {self.pid} LoopCount: {count}')
            self.lock.release()

if __name__ == '__main__':
    lock = Lock()
    for i in range(10, 15):
        p = MyProcess(i, lock)
        p.start()


## 信号量

进程互斥锁，可以使同一时刻只有一个进程能访问共享资源。但是有时我们需要允许多个进程来访问共享资源，同时还需要限制能访问共享资源进程的数量。

这种需求可以用信号量实现。信号量使进程同步过程中一个比较重要的角色。它可以控制临界资源的数量，实现多个进程同时访问共享资源，限制进程的并发量。

In [11]:
from multiprocessing import Process, Semaphore, Lock, Queue
import time

buffer = Queue(10)    # 定义一个共享队列
empty = Semaphore(2)  # 定义信号量，代表缓冲区空余数
full = Semaphore(0)   # 定义信号量，代表缓冲区占用数
lock = Lock()         

class Consumer(Process):
    def run(self):
        global buffer, empty, full, lock
        while True:
            full.acquire() # 使用acquire 占用一个缓冲区位置，缓冲区空余数减一
            lock.acquire() # 加锁
            buffer.get()   # 对缓冲区进行操作
            print('Consumer pop an element')
            time.sleep(1)
            lock.release() # 释放锁
            empty.release()# 

class Producer(Process):
    def run(self):
        global buffer, empty, full, lock
        while True:
            empty.acquire()
            lock.acquire()
            buffer.put(1)
            print('Producer append an element')
            time.sleep(1)
            lock.release()
            full.release()

if __name__ == '__main__':
    p = Producer()
    c = Consumer()
    p.daemon = c.daemon = True
    p.start()
    c.start()
    p.join()
    c.join()
    print('Main Process Ended')


Main Process Ended


## 队列(queue)

上面的例子中，我们使用queue作为进程通信的共享队列。普通list是起不到效果的。因为进程和进程之间的资源是不共享的。即使在一个进程中改变了这个 list，在另一个进程也不能获取到这个 list 的状态，所以声明全局变量对多进程是没有用处的。

那么进程共享数据是通过`multiprocessing`的`Queue`实现的



In [12]:
from multiprocessing import Process, Semaphore, Lock, Queue
import time
from random import random

buffer = Queue(10)
empty = Semaphore(2)
full = Semaphore(0)
lock = Lock()

class Consumer(Process):
    def run(self):
        global buffer, empty, full, lock
        while True:
            full.acquire()
            lock.acquire()
            print(f'Consumer get {buffer.get()}')
            time.sleep(1)
            lock.release()
            empty.release()

class Producer(Process):
    def run(self):
        global buffer, empty, full, lock
        while True:
            empty.acquire()
            lock.acquire()
            num = random()
            print(f'Producer put {num}')
            buffer.put(num)
            time.sleep(1)
            lock.release()
            full.release()

if __name__ == '__main__':
    p = Producer()
    c = Consumer()
    p.daemon = c.daemon = True
    p.start()
    c.start()
    p.join()
    c.join()
    print('Main Process Ended')


Main Process Ended


在上面的例子中我们声明了两个进程，一个进程为生产者 Producer，另一个为消费者 Consumer，生产者不断向 Queue 里面添加随机数，消费者不断从队列里面取随机数。

生产者在放数据的时候调用了 Queue 的 put 方法，消费者在取的时候使用了 get 方法，这样我们就通过 Queue 实现两个进程的数据共享了。

## 管道

实现进程之间直接通信，可以用Pipe，管道。

管道，我们可以把它理解为两个进程之间通信的通道。管道可以是单向的，即 half-duplex：一个进程负责发消息，另一个进程负责收消息；也可以是双向的 duplex，即互相收发消息。

默认声明 Pipe 对象是双向管道，如果要创建单向管道，可以在初始化的时候传入 deplex 参数为 False。

In [13]:
from multiprocessing import Process, Pipe

class Consumer(Process):
    def __init__(self, pipe):
        Process.__init__(self)
        self.pipe = pipe

    def run(self):
        self.pipe.send('Consumer Words')
        print(f'Consumer Received: {self.pipe.recv()}')

class Producer(Process):
    def __init__(self, pipe):
        Process.__init__(self)
        self.pipe = pipe

    def run(self):
        print(f'Producer Received: {self.pipe.recv()}')
        self.pipe.send('Producer Words')

if __name__ == '__main__':
    pipe = Pipe()
    p = Producer(pipe[0])
    c = Consumer(pipe[1])
    p.daemon = c.daemon = True
    p.start()
    c.start()
    p.join()
    c.join()
    print('Main Process Ended')


Main Process Ended


在这个例子里我们声明了一个默认为双向的管道，然后将管道的两端分别传给两个进程。两个进程互相收发。

管道 Pipe 就像进程之间搭建的桥梁，利用它我们就可以很方便地实现进程间通信了。

## 进程池

在前面，我们讲了可以使用 Process 来创建进程，同时也讲了如何用 Semaphore 来控制进程的并发执行数量。

假如现在我们遇到这么一个问题，我有 10000 个任务，每个任务需要启动一个进程来执行，并且一个进程运行完毕之后要紧接着启动下一个进程，同时我还需要控制进程的并发数量，不能并发太高，不然 CPU 处理不过来（如果同时运行的进程能维持在一个最高恒定值当然利用率是最高的）。

那么我们该如何来实现这个需求呢？

用 Process 和 Semaphore 可以实现，但是实现起来比较我们可以用 Process 和 Semaphore 解决问题，但是实现起来比较烦琐。而这种需求在平时又是非常常见的。此时，我们就可以派上进程池了，即 multiprocessing 中的 Pool。

Pool 可以提供指定数量的进程，供用户调用，当有新的请求提交到 pool 中时，如果池还没有满，就会创建一个新的进程用来执行该请求；但如果池中的进程数已经达到规定最大值，那么该请求就会等待，直到池中有进程结束，才会创建新的进程来执行它。

我们用一个实例来实现一下，代码如下：

In [ ]:
from multiprocessing import Pool
import time


def function(index):
    print(f'Start process: {index}')
    time.sleep(3)
    print(f'End process {index}', )


if __name__ == '__main__':
    pool = Pool(processes=3)                    # 声明了一个大小为3的进程池
    for i in range(4):
        pool.apply_async(function, args=(i,))   # 使用apply_async方法将进程添加进去

    print('Main Process started')
    pool.close()                                # 调用close方法来关闭进程池，使其不再接受新的任务
    pool.join()                                 # 调用join方法让主进程等待子进程的退出
    print('Main Process ended')


In [ ]:
# 一个更加简洁的实现：使用map方法

from multiprocessing import Pool
import urllib.request
import urllib.error


def scrape(url):
    try:
        urllib.request.urlopen(url)
        print(f'URL {url} Scraped')
    except (urllib.error.HTTPError, urllib.error.URLError):
        print(f'URL {url} not Scraped')


if __name__ == '__main__':
    pool = Pool(processes=3)
    urls = [
        'https://www.baidu.com',
        'http://www.meituan.com/',
        'http://blog.csdn.net/',
        'http://xxxyxxx.net'
    ]
    pool.map(scrape, urls)      # 第一个参数是启动的进程对应的执行方法，第二个参数是一个可迭代对象
    pool.close()


这个例子中我们先定义了一个 scrape 方法，它接收一个参数 url，这里就是请求了一下这个链接，然后输出爬取成功的信息，如果发生错误，则会输出爬取失败的信息。

首先我们要初始化一个 Pool，指定进程数为 3。然后我们声明一个 urls 列表，接着我们调用了 map 方法，第 1 个参数就是进程对应的执行方法，第 2 个参数就是 urls 列表，map 方法会依次将 urls 的每个元素作为 scrape 的参数传递并启动一个新的进程，加到进程池中执行。

这样，我们就可以实现 3 个进程并行运行。不同的进程相互独立地输出了对应的爬取结果。

可以看到，我们利用 Pool 的 map 方法非常方便地实现了多进程的执行。后面我们也会在实战案例中结合进程池来实现数据的爬取。

以上便是 Python 中多进程的基本用法，本节内容比较多，后面的实战案例也会用到这些内容，需要好好掌握。